# OpenCV Face Recognition

In [47]:
#import OpenCV module
import cv2
#import os module for reading training data directories and paths
import os
#import numpy to convert python lists to numpy arrays as 
#it is needed by OpenCV face recognizers
import numpy as np

#matplotlib for display our images
import matplotlib.pyplot as plt
%matplotlib inline 

## Prepare training data

In [130]:
#function to detect face using OpenCV
def detect_face(img):
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')
    
    #img_copy = np.copy(colored_img)
    
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #gray = np.array(gray, "uint8")
    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h]

In [131]:
#this function will read all persons' training images, detect face from each image
#and will return two lists of exactly same size, one list 
# of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
    
    #list the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name
            print("adding image path: ", image_path)
            
            #read image
            image = cv2.imread(image_path)
            
            #detect face
            face = detect_face(image)
            if face is not None:
                print("Face detected: ", face.shape)
                
            cv2.imshow("Adding training image...", image)
            cv2.waitKey(100)
            
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [132]:
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

# img = cv2.imread("training-data/s1/4.pgm")
# plt.imshow(img)

# face = detect_face(img)
# if face is not None:
#     plt.imshow(face)
# else:
#     print("No face detected")

Preparing data...
adding image path:  training-data/s1/1.pgm
Face detected:  (82, 82)
adding image path:  training-data/s1/10.pgm
Face detected:  (61, 61)
adding image path:  training-data/s1/2.pgm
adding image path:  training-data/s1/3.pgm
Face detected:  (79, 79)
adding image path:  training-data/s1/4.pgm
adding image path:  training-data/s1/5.pgm
adding image path:  training-data/s1/6.pgm
adding image path:  training-data/s1/7.pgm
Face detected:  (86, 86)
adding image path:  training-data/s1/8.pgm
Face detected:  (81, 81)
adding image path:  training-data/s1/9.pgm
Face detected:  (83, 83)
adding image path:  training-data/s2/1.pgm
Face detected:  (85, 85)
adding image path:  training-data/s2/10.pgm
Face detected:  (76, 76)
adding image path:  training-data/s2/2.pgm
adding image path:  training-data/s2/3.pgm
Face detected:  (82, 82)
adding image path:  training-data/s2/4.pgm
Face detected:  (87, 87)
adding image path:  training-data/s2/5.pgm
adding image path:  training-data/s2/6.pgm

In [128]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.createLBPHFaceRecognizer()

In [129]:
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

In [146]:
img = cv2.imread("training-data/s4/4.pgm")
face = detect_face(img)

label= face_recognizer.predict(face)
print(label)
#print("Predicted: label, confidence", label, confidence)

4
